### Using CrewAI with langchains langraph
Langgraph is a new feature for orchestrating multi agent behaviour with langchain, we will be experiment using langgraph together with the capabilities provided by crew ai

##### Tool definition
The first step in building an agent crew with crew ai is to setup the tools, in this example we would be setting up a tool for drafting emails with langchain


In [1]:
from langchain_community.agent_toolkits import GmailToolkit
from langchain_community.tools.gmail.create_draft import GmailCreateDraft
from langchain.tools import tool

In [2]:
class CreateDraftTool():
    
    @tool("Create Draft")
    def create_draft(data:str):
        """ 
        Useful to create an email draft.
        The input to this tool should be a pipe (|) separated text
        of length 3 (three), representing who to send the email to,
        the subject of the email and the actual message.
        For example, `lorem@ipsum.com|Nice To Meet You|Hey it was great to meet you.`.
        """
        email, subject, message = data.split("|")
        gmail = GmailToolkit()
        draft = GmailCreateDraft(api_resource=gmail.api_resource)
        result = draft({
				'to': [email],
				'subject': subject,
		'message': message
		})
        return f"\nDraft created: {result}\n"

In [3]:
from textwrap import dedent
from crewai import Agent

In [4]:
from langchain_community.agent_toolkits import GmailToolkit
from langchain_community.tools.gmail.get_thread import GmailGetThread
from langchain_community.tools.tavily_search import TavilySearchResults
from textwrap import dedent

In [5]:
class EmailReviewAgent():
    def __init__(self):
        self.gmail = GmailToolkit()
        
    def email_filter_agent(self):
        return Agent(
            role="Senior Email Analyst",
            goal="Filter out non-essential emails like newsletters and promotional content",
            backstory=dedent(
                """
                As a Senior Email Analyst, you have extensive experience in email content analysis.
				You are adept at distinguishing important emails from spam, newsletters, and other
				irrelevant content. Your expertise lies in identifying key patterns and markers that
				signify the importance of an email.
                """),
            verbose=True,
            allow_delegation=False
        )
        
    def email_action_agent(self):
        return Agent(
            role="Email Action Specialist",
            goal="Identify action-required emails and compile a list of their IDs",
            backstory=dedent(
                """
                With a keen eye for detail and a knack for understanding context, you specialize
				in identifying emails that require immediate action. Your skill set includes interpreting
				the urgency and importance of an email based on its content and context.
                """),
            tools=[
                GmailGetThread(api_resource=self.gmail.api_resource),
				TavilySearchResults()
                ],
            allow_delegation=False,
            verbose=True, 
        )
        
        
    def email_resposne_writer(self):
        return Agent(
            role="Email Response Writer",
            goal="Draft responses to action-required emails",
            backstory=dedent(
                """
                You are a skilled writer, adept at crafting clear, concise, and effective email responses.
				Your strength lies in your ability to communicate effectively, ensuring that each response is
				tailored to address the specific needs and context of the email.
                """),
            tools=[
                TavilySearchResults(),
				GmailGetThread(api_resource=self.gmail.api_resource),
				CreateDraftTool.create_draft
                ],
            verbose=True,
            allow_delegation=False
        )

#### Define Tasks
Now we have defined the tools and the agents, we can now define the tasks that will be performed by each indivual agent to accomplish the overarching goal

In [6]:
from crewai import Task

In [7]:
class EmailFilterTasks:
    def filter_email_task(self, agent, emails):
        return Task(
            description=dedent(
            f"""
            Analyze a batch of emails and filter out
			non-essential ones such as newsletters, promotional content and notifications.

			Use your expertise in email content analysis to distinguish
			important emails from the rest, pay attention to the sender and avoind invalid emails.

			Make sure to filter for the messages actually directed at the user and avoid notifications.

			EMAILS
			-------
			{emails}

			Your final answer MUST be a the relevant thread_ids and the sender, use bullet points.
            """
            ),
            agent=agent
        )
    
    def action_required_email_task(self, agent):
        return Task(
            description=dedent("""
				For each email thread, pull and analyze the complete threads using only the actual Thread ID.
				understand the context, key points, and the overall sentiment
				of the conversation.

				Identify the main query or concerns that needs to be
				addressed in the response for each

				Your final answer MUST be a list for all emails with:
				- the thread_id
				- a summary of the email thread
				- a highlighting with the main points
				- identify the user and who he will be answering to
				- communication style in the thread
				- the sender's email address
				"""),
			agent=agent
        )
    
    def draft_response_task(self, agent):
        return Task(
            description=dedent(f"""\
				Based on the action-required emails identified, draft responses for each.
				Ensure that each response is tailored to address the specific needs
				and context outlined in the email.

				- Assume the persona of the user and mimic the communication style in the thread.
				- Feel free to do research on the topic to provide a more detailed response, IF NECESSARY.
				- IF a research is necessary do it BEFORE drafting the response.
				- If you need to pull the thread again do it using only the actual Thread ID.

				Use the tool provided to draft each of the responses.
				When using the tool pass the following input:
				- to (sender to be responded)
				- subject
				- message

				You MUST create all drafts before sending your final answer.
				Your final answer MUST be a confirmation that all responses have been drafted.
				"""),
			agent=agent
        )

#### Define the crew
Now that the tasks are defined, we can now define the crew operation for performing these tasks.

In [8]:
from crewai import Crew

In [9]:
class EmailFilterCrew():
    def __init__(self):
        agents = EmailReviewAgent()
        self.filter_agent = agents.email_filter_agent()
        self.action_agent = agents.email_action_agent()
        self.writer_agent = agents.email_resposne_writer()
    
    def kickoff(self, state):
        print("###FILTERING EMAILS###")
        tasks = EmailFilterTasks()
        crew = Crew(
            agents=[],
            tasks=[
                tasks.filter_email_task(self.filter_agent, self._format_emails(state['emails'])),
                tasks.action_required_email_task(self.action_agent),
                tasks.draft_response_task(self.writer_agent)
            ]
        )
        
        result = crew.kickoff()
        return {**state, "action_required_emails": result}
    
    
    def _format_emails(self, emails):
        emails_string = []
        for email in emails:
            print(email)
            arr = [
				f"ID: {email['id']}",
				f"- Thread ID: {email['threadId']}",
				f"- Snippet: {email['snippet']}",
				f"- From: {email['sender']}",
				f"--------"
			]
            emails_string.append("\n".join(arr))
        return "\n".join(emails_string)

#### Defining a StateGraph with langgraph
This should be interesting

In [10]:
from langgraph.graph import StateGraph


In [12]:
import time
from langchain_community.tools.gmail.search import GmailSearch
import os

In [13]:
class Nodes():
    def __init__(self):
        self.gmail = GmailToolkit()
    
    def check_email(self, state):
        print("Checking for new emails")
        search = GmailSearch(api_resource=self.gmail.api_resource)
        emails = search('after:newer_than:1d')
        checked_emails = state['checked_emails_ids'] if state['checked_emails_ids'] else []
        thread = []
        new_emails = []
        for email in emails:
            if (email['id'] not in checked_emails) and (email['threadId'] not in thread) and ( os.environ['MY_EMAIL'] not in email['sender']):
                thread.append(email['threadId'])
                new_emails.append({"id": email['id'],"threadId": email['threadId'],"snippet": email['snippet'],"sender": email["sender"]})
        
        checked_emails.extend([email['id'] for email in emails])
        return {**state,"emails": new_emails,"checked_emails_ids": checked_emails}
    
    def wait_next_run(self, state):
        print("waiting for 180 seconds")
        time.sleep(180)
        return state
        
        
    def new_emails(self, state):
        if len(state['emails']) == 0:
            print("No new emails")
            return "end"
        else:
            print("new emails")
            return "continue"

In [14]:
import datetime
from typing import TypedDict

class EmailsState(TypedDict):
    checked_emails_ids: list[str]
    emails: list[dict]
    action_required_emails: dict

In [15]:
class WorkFlow():
    def __init__(self):
        nodes = Nodes()
        workflow = StateGraph(EmailsState)
        workflow.add_node("check_new_emails", nodes.check_email)
        workflow.add_node("wait_next_run", nodes.wait_next_run)
        workflow.add_node("draft_responses", EmailFilterCrew().kickoff)
        
        workflow.set_entry_point("check_new_emails")
        workflow.add_conditional_edges(
				"check_new_emails",
				nodes.new_emails,
				{
					"continue": 'draft_responses',
					"end": 'wait_next_run'
				}
		)
        
        workflow.add_edge('draft_responses', 'wait_next_run')
        workflow.add_edge('wait_next_run', 'check_new_emails')
        self.app = workflow.compile()
        

#### Putting everything together
We are using the crew as single node withing the workflow

In [ ]:
app = WorkFlow().app
app.invoke({})